# HERE Routing API
[Routing API v8](https://developer.here.com/documentation/routing-api/8.20.3/dev_guide/index.html)

<div class="alert alert-block alert-warning">
	<b>Warning:</b> Before running below cells please make sure you have API key. 
                Please see 
	<a href="https://github.com/heremaps/here-location-services-python/blob/master/docs/notebooks/README.md">README.md</a> for more info on API key.

</div>

In [ ]:
import os
from datetime import datetime

from here_location_services import LS
from here_location_services.config.routing_config import ROUTING_RETURN, Via
from here_location_services.config.matrix_routing_config import Truck, SHIPPED_HAZARDOUS_GOODS
from here_map_widget import Map, Marker, GeoJSON, DomMarker, DomIcon

In [ ]:
# Create Location Services object using API KEY.

LS_API_KEY = os.environ.get("LS_API_KEY")  # Get API KEY from environment.

ls = LS(api_key=LS_API_KEY)

## Calculate Route
A basic route request consists of an origin, destination. The resulting response will contain a list of routes. The list may be empty if no routes were possible.

The origin and destination can be a latitude and longitude pairs and represent the beginning and end of the requested route.

If the request specifies alternatives, the response may provide additional routes.

If the request specifies return with summary or travel_summary, then the sections will provide additional summary information such as length and duration.

A route will include one or more sections.

### Car Route
Calculate car route between origin and destination.

In [ ]:
result = ls.car_route(
    origin=[52.51375, 13.42462],
    destination=[52.52332, 13.42800],
    return_results=[
        ROUTING_RETURN.polyline,
        ROUTING_RETURN.elevation,
        ROUTING_RETURN.instructions,
        ROUTING_RETURN.actions,
    ],
)

In [ ]:
geo_json = result.to_geojson()
data = geo_json
geo_layer = GeoJSON(data=data, style={"lineWidth": 5})

m = Map(api_key=LS_API_KEY, center=[52.5207, 13.4283], zoom=14)
origin_marker = Marker(lat=52.51375, lng=13.42462)
dest_marker = Marker(lat=52.52332, lng=13.42800)
m.add_layer(geo_layer)
m.add_object(origin_marker)
m.add_object(dest_marker)
m

## Route via Intermediate Waypoint

For a detour from the best route, specify the via parameter in your request.

For a list of via waypoints, the resulting route looks as follows:

1. from the origin to the first via
2. to any subsequent via in the requested order
3. the last via until the destination

In [ ]:
result = ls.car_route(
    origin=[52.51375, 13.42462],
    destination=[52.52332, 13.42800],
    via=[Via(lat=52.52426, lng=13.43000)],
    return_results=[
        ROUTING_RETURN.polyline,
        ROUTING_RETURN.elevation,
        ROUTING_RETURN.instructions,
        ROUTING_RETURN.actions,
    ],
)

geo_json = result.to_geojson()
data = geo_json
geo_layer = GeoJSON(data=data, style={"lineWidth": 5})

m = Map(api_key=LS_API_KEY, center=[52.5207, 13.4283], zoom=14)
origin_marker = Marker(lat=52.51375, lng=13.42462)
dest_marker = Marker(lat=52.52332, lng=13.42800)
via_marker = Marker(lat=52.52426, lng=13.43000)
m.add_layer(geo_layer)
m.add_object(origin_marker)
m.add_object(dest_marker)
m.add_object(via_marker)
m

## Route to the correct side of street.

A taxi-driver would like to drop off the passenger "at the doorstep". This requires the route to go the correct side of the street. To arrive at the destination on the correct side-of-street, specify the sideOfStreetHint parameter for the destination_place_options.

sideOfStreetHint accepts latitude, longitude as input. The value expected to be used for this, normally, is the position of the PoI (Point of Interest) beside the road. The resulting route will try to take this hint into account and route you to the correct side of the street.

In [ ]:
from here_location_services.config.routing_config import PlaceOptions

dpo = PlaceOptions(sideof_street_hint=[52.500541,13.363060])

result = ls.car_route(
    origin=[52.511985,13.240523],
    destination=[52.500541,13.363060],
    return_results=[
        ROUTING_RETURN.polyline,
        ROUTING_RETURN.elevation,
        ROUTING_RETURN.summary
    ],
    destination_place_options=dpo
)

In [ ]:
geo_json = result.to_geojson()
data = geo_json
geo_layer = GeoJSON(data=data, style={"lineWidth": 5, "strokeColor": "purple"})

m = Map(api_key=LS_API_KEY, center=[52.5076, 13.3451], zoom=14)
icon1 = DomIcon(
    element='<div style="user-select: none; cursor: default;"> \
<div style="color: red; background-color: blue; border: 2px solid black;'
    + "font: 12px / 12px arial; padding-top: 2px; padding-left: 4px; width: 20px;"
    + 'height: 20px; margin-top: -10px; margin-left: -10px;">O</div> \
</div>'
)
icon2 = DomIcon(
    element='<div style="user-select: none; cursor: default;"> \
<div style="color: red; background-color: blue; border: 2px solid black;'
    + "font: 12px / 12px arial; padding-top: 2px; padding-left: 4px; width: 20px;"
    + 'height: 20px; margin-top: -10px; margin-left: -10px;">D</div> \
</div>'
)
origin_marker = DomMarker(lat=52.511985, lng=13.240523, icon=icon1)
dest_marker = DomMarker(lat=52.500541, lng=13.363060, icon=icon2)
m.add_objects([origin_marker, dest_marker])
m.add_layer(geo_layer)

In [ ]:
result2 = ls.car_route(
    origin=[52.511985,13.240523],
    destination=[52.500541,13.363060],
    return_results=[
        ROUTING_RETURN.polyline,
        ROUTING_RETURN.elevation,
        ROUTING_RETURN.summary
    ],
)

In [ ]:
geo_json2 = result2.to_geojson()
data2 = geo_json2
geo_layer2 = GeoJSON(data=data2, style={"lineWidth": 5, "strokeColor": "blue"})
m.add_layer(geo_layer2)
m

In the above example we can see that purple route is using `sideof_street_hint` using destination place options and blue route is without using it.

## Truck Routing
The Truck Routing features of the API extend router with truck-specific options.

In [ ]:
truck = Truck(
    shipped_hazardous_goods=[
        SHIPPED_HAZARDOUS_GOODS.combustible,
        SHIPPED_HAZARDOUS_GOODS.explosive,
    ],
    gross_weight=4000,
)

result = ls.truck_route(
    origin=[50.892387, 20.655124],
    destination=[50.894911, 20.64899],
    departure_time=datetime(2020, 10, 28, 0, 1, 0),
    truck=truck,
)

In [ ]:
result.routes